In [19]:
import pandas as pd
df_train = pd.read_csv('df_train_sbert.csv')
df_test = pd.read_csv('df_test_sbert.csv')
df_train_label = df_train[df_train['label'] == True]
df_test_label = df_test[df_test['label'] == True]

In [20]:
import numpy as np

# Function to safely convert string embeddings to numpy arrays
def parse_embedding(embedding):
    if isinstance(embedding, str):  # If stored as a string
        embedding = embedding.replace("[", "").replace("]", "")  # Remove brackets
        return np.array([float(x) for x in embedding.split()])  # Convert to float array
    return np.array(embedding)  # If already numeric, keep as is

# Convert all embeddings properly
df_train_label["sbert_embedding"] = df_train_label["sbert_embedding"].apply(parse_embedding)
df_test_label["sbert_embedding"] = df_test_label["sbert_embedding"].apply(parse_embedding)


/tmp/ipykernel_4291/221399675.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_label["sbert_embedding"] = df_train_label["sbert_embedding"].apply(parse_embedding)
/tmp/ipykernel_4291/221399675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_label["sbert_embedding"] = df_test_label["sbert_embedding"].apply(parse_embedding)


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Concatenate the existing training and test sets into one DataFrame.
df_all_label = pd.concat([df_train, df_test], ignore_index=True)

# Now, re-split the combined dataset using stratification to maintain class distribution.
# Replace 'topic_id' with 'label' if it's a binary task.
df_train_label, df_test_label = train_test_split(df_all_label, test_size=0.2, random_state=42, stratify=df_all_label['label'])

# Optionally, reset the index for convenience.
df_train_label = df_train_label.reset_index(drop=True)
df_test_label = df_test_label.reset_index(drop=True)

# Check the new class distributions to ensure stratification worked as intended.
print("Train class distribution:")
print(df_train_label['topic_id'].value_counts(normalize=True))
print("\nTest class distribution:")
print(df_test_label['topic_id'].value_counts(normalize=True))


Train class distribution:
topic_id
547.0    0.103959
550.0    0.103724
544.0    0.102848
546.0    0.102591
554.0    0.102356
552.0    0.102356
543.0    0.101780
602.0    0.101694
556.0    0.100776
600.0    0.077916
Name: proportion, dtype: float64

Test class distribution:
topic_id
602.0    0.106128
543.0    0.105786
552.0    0.103475
554.0    0.103475
546.0    0.102533
544.0    0.101506
550.0    0.097997
556.0    0.097398
547.0    0.097056
600.0    0.084646
Name: proportion, dtype: float64


In [15]:
df_train_label = df_train[df_train['label'] == True]
df_test_label = df_test[df_test['label'] == True]

print("Train class distribution:")
print(df_train_label['topic_id'].value_counts(normalize=True))
print("\nTest class distribution:")
print(df_test_label['topic_id'].value_counts(normalize=True))

Train class distribution:
topic_id
602.0    0.175889
543.0    0.154646
546.0    0.150320
544.0    0.148318
550.0    0.145864
547.0    0.100342
600.0    0.058824
554.0    0.028863
556.0    0.019177
552.0    0.017757
Name: proportion, dtype: float64

Test class distribution:
topic_id
602.0    0.175961
550.0    0.152788
543.0    0.149223
546.0    0.145913
544.0    0.141839
547.0    0.110772
600.0    0.058569
554.0    0.029794
552.0    0.018844
556.0    0.016297
Name: proportion, dtype: float64


above some sanity check. Some topic just have few true labels.

In [6]:
# Convert SBERT embeddings to 2D NumPy arrays
X_train = np.vstack(df_train_label['sbert_embedding'].values)
X_test  = np.vstack(df_test_label['sbert_embedding'].values)

# Use 'topic_id' as labels for multiclass classification
y_train = df_train_label['topic_id'].values
y_test  = df_test_label['topic_id'].values

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Set up logistic regression for multiclass classification
lr = LogisticRegression(multi_class='multinomial',
                        solver='lbfgs',
                        max_iter=1000,
                        class_weight='balanced')

# Define a grid for the regularization parameter C
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

# Use GridSearchCV to find the best C based on macro F1 score
grid_search = GridSearchCV(lr, param_grid, scoring='f1_macro', cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best logistic regression model
best_lr = grid_search.best_estimator_
print(f"Best C: {grid_search.best_params_['C']}")

# Predict on the test set
y_pred = best_lr.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Fitting 3 folds for each of 5 candidates, totalling 15 fits


/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and wi

[CV] END .............................................C=0.01; total time=  12.0s
[CV] END .............................................C=0.01; total time=  14.2s
[CV] END .............................................C=0.01; total time=  11.6s
[CV] END ..............................................C=0.1; total time=  19.6s
[CV] END ..............................................C=0.1; total time=  24.0s
[CV] END ..............................................C=0.1; total time=  22.5s
[CV] END ................................................C=1; total time=  39.7s
[CV] END ................................................C=1; total time=  45.2s
[CV] END ................................................C=1; total time=  43.4s
[CV] END ...............................................C=10; total time= 1.1min
[CV] END ...............................................C=10; total time= 1.2min
[CV] END ...............................................C=10; total time= 1.3min
[CV] END ...................

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Convert each embedding to a NumPy array of type float32 before stacking
X_train = np.vstack(df_train_label['sbert_embedding'].apply(lambda x: np.array(x, dtype=np.float32)))
X_test  = np.vstack(df_test_label['sbert_embedding'].apply(lambda x: np.array(x, dtype=np.float32)))

# Optionally, check the dtype and shape
print("X_train dtype:", X_train.dtype, "shape:", X_train.shape)
print("X_test dtype:", X_test.dtype, "shape:", X_test.shape)


# Get original labels
y_train_orig = df_train_label['topic_id'].values
y_test_orig  = df_test_label['topic_id'].values

# Create mapping from original labels to new labels (0, 1, 2, ...)
unique_labels = np.sort(np.unique(y_train_orig))
mapping = {label: idx for idx, label in enumerate(unique_labels)}

# Map the labels
y_train = np.array([mapping[label] for label in y_train_orig])
y_test  = np.array([mapping[label] for label in y_test_orig])

# Determine the number of classes (should equal len(unique_labels))
num_classes = len(unique_labels)
print("Unique labels:", unique_labels)
print("Number of classes:", num_classes)

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2]
}

# Initialize the XGBoost classifier for multiclass classification.
xgb_clf = XGBClassifier(
    objective='multi:softmax',
    num_class=num_classes,
    eval_metric='mlogloss',
    random_state=42,
    n_jobs=-1
)

# Set up GridSearchCV with macro F1 score as the metric.
grid_search = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Display the best parameters and score
print("Best parameters found:", grid_search.best_params_)
print("Best macro F1 score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


X_train dtype: float32 shape: (15487, 384)
X_test dtype: float32 shape: (3927, 384)
Unique labels: [543. 544. 546. 547. 550. 552. 554. 556. 600. 602.]
Number of classes: 10
Fitting 3 folds for each of 81 candidates, totalling 243 fits


In [25]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Convert each embedding to a NumPy array of type float32 before stacking
X_train_full = np.vstack(df_train_label['sbert_embedding'].apply(lambda x: np.array(x, dtype=np.float32)))
X_test  = np.vstack(df_test_label['sbert_embedding'].apply(lambda x: np.array(x, dtype=np.float32)))

# Check dtype and shape
print("X_train_full dtype:", X_train_full.dtype, "shape:", X_train_full.shape)
print("X_test dtype:", X_test.dtype, "shape:", X_test.shape)

# Get original labels
y_train_orig = df_train_label['topic_id'].values
y_test_orig  = df_test_label['topic_id'].values

# Create mapping from original labels to new contiguous labels (0, 1, 2, ...)
unique_labels = np.sort(np.unique(y_train_orig))
mapping = {label: idx for idx, label in enumerate(unique_labels)}
y_train_full = np.array([mapping[label] for label in y_train_orig])
y_test  = np.array([mapping[label] for label in y_test_orig])

# Determine number of classes
num_classes = len(unique_labels)
print("Unique labels:", unique_labels)
print("Number of classes:", num_classes)

# **Reduce training size by sampling a subset (e.g., 20% of the full training data)**
subset_fraction = 0.5  # Use 20% of training data
X_train, _, y_train, _ = train_test_split(
    X_train_full, y_train_full, 
    test_size=1 - subset_fraction, 
    random_state=42,
    stratify=y_train_full
)

print("Reduced X_train shape:", X_train.shape)
print("Reduced y_train shape:", y_train.shape)

# Define a smaller parameter grid for RandomizedSearchCV
param_dist = {
    'max_depth': [3, 5, 10],              
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200],
    'gamma': [0, 0.1]
}

# Initialize the XGBoost classifier for multiclass classification.
xgb_clf = XGBClassifier(
    objective='multi:softmax',
    num_class=num_classes,
    eval_metric='mlogloss',
    random_state=42,
    n_jobs=-1
)

# Set up RandomizedSearchCV with fewer iterations and 2-fold CV.
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=10,              # try 10 random combinations
    scoring='f1_macro',
    cv=3,                   # use 3-fold CV to speed up
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Perform the randomized search on the reduced training set
random_search.fit(X_train, y_train)

# Display best parameters and score
print("Best parameters found:", random_search.best_params_)
print("Best macro F1 score:", random_search.best_score_)

# Evaluate the best model on the full test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



X_train_full dtype: float32 shape: (15487, 384)
X_test dtype: float32 shape: (3927, 384)
Unique labels: [543. 544. 546. 547. 550. 552. 554. 556. 600. 602.]
Number of classes: 10
Reduced X_train shape: (7743, 384)
Reduced y_train shape: (7743,)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters found: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.1}
Best macro F1 score: 0.618881277103864
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.74      0.66       586
           1       0.39      0.36      0.38       557
           2       0.79      0.80      0.80       573
           3       0.86      0.62      0.72       435
           4       0.65      0.77      0.70       600
           5       0.84      0.28      0.42        74
           6       0.68      0.35      0.46       117
           7       0.75      0.47      0.58        64
           8       0.88      0.72      0.79      

In [29]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Assume df_train_label has:
# - 'sbert_embedding': SBERT embeddings (each a list/array)
# - 'topic_id': multiclass labels (e.g., 543.0, 544.0, etc.)

# Convert embeddings to a 2D NumPy array of type float32
X_full = np.vstack(df_train_label['sbert_embedding'].apply(lambda x: np.array(x, dtype=np.float32)))
y_orig = df_train_label['topic_id'].values

# Remap original labels to contiguous integers (0, 1, 2, ...)
unique_labels = np.sort(np.unique(y_orig))
mapping = {label: idx for idx, label in enumerate(unique_labels)}
y_full = np.array([mapping[label] for label in y_orig])

print("Full training set shape:", X_full.shape)
print("Unique labels:", unique_labels)

# Reduce training size by sampling a subset (e.g., 20% of the data) using stratification.
subset_fraction = 0.5
X_subset, _, y_subset, _ = train_test_split(
    X_full, y_full,
    test_size=1 - subset_fraction,
    random_state=42,
    stratify=y_full
)
print("Reduced training set shape:", X_subset.shape)

# Define a parameter grid that explores both linear and RBF kernels.
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1]}
]

# Initialize the SVM classifier.
svm = SVC(random_state=42)

# Set up GridSearchCV using 5-fold cross-validation and macro F1 as the scoring metric.
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Perform the grid search on the subset.
grid_search.fit(X_subset, y_subset)

# Display the best parameters and best macro F1 score.
print("Best parameters found:", grid_search.best_params_)
print("Best macro F1 score:", grid_search.best_score_)

# Optionally, evaluate on the full test set if available:
# Assuming you have a similar test set, you can process it as follows:
# (Here, we assume df_test_label exists similarly with 'sbert_embedding' and 'topic_id'.)
X_test = np.vstack(df_test_label['sbert_embedding'].apply(lambda x: np.array(x, dtype=np.float32)))
y_test_orig = df_test_label['topic_id'].values
y_test = np.array([mapping[label] for label in y_test_orig])

y_pred = grid_search.best_estimator_.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Full training set shape: (15487, 384)
Unique labels: [543. 544. 546. 547. 550. 552. 554. 556. 600. 602.]
Reduced training set shape: (7743, 384)
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best parameters found: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Best macro F1 score: 0.684086936436397
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.80      0.70       586
           1       0.49      0.39      0.44       557
           2       0.82      0.82      0.82       573
           3       0.88      0.66      0.76       435
           4       0.67      0.80      0.73       600
           5       0.71      0.47      0.57        74
           6       0.68      0.50      0.58       117
           7       0.78      0.67      0.72        64
           8       0.89      0.81      0.85       230
           9       0.83      0.87      0.85       691

    accuracy                           0.72      3927
   macro avg     